## Imports and Setup

In [ ]:
import os
import sys

import arviz as av
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import yaml

from threeML import update_logging_level

update_logging_level("FATAL")

parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from zusammen import DataSet

sns.set_theme(context="paper")

mpl.use("pgf")
pgf_with_latex = {
    "text.usetex": True,
    "font.family": "serif",
    "axes.labelsize": 10,
    "font.size": 10,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "pgf.texsystem": "lualatex",
    "pgf.preamble": "\n".join([r"\usepackage{siunitx}", r"\DeclareSIUnit{\erg}{erg}"]),
}
mpl.rcParams.update(pgf_with_latex)

width = 455

In [ ]:
simulation_folder = "simulation/"
survey_name = "survey"

inference_folder = "inference/"
data_name = "data_sig_5"

inference_name = "relaxed_sig_5_1000"

In [ ]:
ds = DataSet.from_hdf5_file(inference_folder + data_name + ".h5")
data = ds.to_stan_dict()
res = av.from_netcdf(inference_folder + inference_name + ".nc")

#### Set size

In [ ]:
def pt_to_inch(pt):
    return pt / 72.27


def set_size(width_pt, fraction=1, subplots=(1, 1), height=None):
    """Set figure dimensions to sit nicely in our document.

    Parameters
    ----------
    width_pt: float
            Document width in points
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**0.5 - 1) / 2

    # Figure height in inches
    if not height:
        fig_height_in = (
            pt_to_inch(fig_width_pt) * golden_ratio * (subplots[0] / subplots[1])
        )
    else:
        fig_height_in = pt_to_inch(height)

    return (pt_to_inch(fig_width_pt), fig_height_in)

## Band function

In [ ]:
def band(E, Ec, piv, K, alpha, beta):
    ret = np.zeros(E.shape)
    for i, e in enumerate(E):
        if e < (alpha - beta) * Ec:
            ret[i] = K * (e / piv) ** alpha * np.exp(-e / Ec)
        else:
            ret[i] = (
                K
                * ((alpha - beta) * Ec / piv) ** (alpha - beta)
                * np.exp(beta - alpha)
                * (e / piv) ** beta
            )
    return ret

In [ ]:
K = 10
alpha = -1
beta = -2.2
Ec = 1
piv = .1
x1 = 1e-2
x2 = 1e2

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    sharex=True,
    figsize=set_size(width, height=270, subplots=(2, 1)),
    height_ratios=(3, 2),
)

E = np.logspace(np.log(x1), np.log(x2), 100)
ax1.plot(E, K * (E / piv) ** alpha * np.exp(-E / Ec), "b--")
ax1.plot(E, band(E, Ec, piv, K, alpha, beta), "b")

ax1.loglog()
ax1.set_xlim(x1, x2)
ax1.set_ylim(1.5e-6, 1e3)

ax1.text(
    0.98,
    0.88,
    "a)",
    horizontalalignment="right",
    transform=ax1.transAxes,
    bbox={"boxstyle": "square", "facecolor": ax1.get_facecolor(), "edgecolor": "gray"},
)
ax1.set_ylabel(
    r"$B$ [$\si{\per\second\per\centi\meter\squared\per\mega\electronvolt}$]"
)


ax2.plot(E, K * 1.602e-6 * E**2 * (E / piv) ** alpha * np.exp(-E / Ec), "b--")
ax2.plot(E, 1.602e-6 * E**2 * band(E, Ec, piv, K, alpha, beta), "b")

ax2.loglog()
ax2.set_ylim(8e-9, 2e-6)
ax2.text(
    0.98,
    0.82,
    "b)",
    horizontalalignment="right",
    transform=ax2.transAxes,
    bbox={"boxstyle": "square", "facecolor": ax2.get_facecolor(), "edgecolor": "gray"},
)
ax2.set_xlabel(r"Photon Energy $E_\gamma$ $[\si{\mega\electronvolt}]$")
ax2.set_ylabel(r"$E_\gamma^2 B$ [$\si{\erg\per\second\per\centi\meter\squared}$]")


fig.align_ylabels([ax1, ax2])
fig.tight_layout()
# fig.savefig("band.pgf")

## PPC

In [ ]:
from posterior_predictive_check import PPC

p = PPC(data, res)

chain = 0
draws = 100
grb = 0
interval = 1
detector = 1

In [ ]:
cenergies, ppc_counts = p.ppc(chain, draws, grb, interval, detector)
ppc_mu, ppc_1s, ppc_2s = PPC.ppc_summary(ppc_counts)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 1, figsize=set_size(width, height=270))

plt.stairs(data["observed_counts"][interval,detector], cenergies, color="#dd8452")
plt.stairs(ppc_1s[0], cenergies, baseline=ppc_1s[1], fill=True, color="#55a868", alpha=.5)
plt.stairs(ppc_2s[0], cenergies, baseline=ppc_2s[1], fill=True, color="#4c72b0", alpha=.2)

# plt.ylim(.8,1e2)
plt.xlabel("Energy")
plt.ylabel("Count Rate")
plt.loglog()

## Light Curves

In [ ]:
from cosmogrb.universe.survey import Survey

survey = Survey.from_file(simulation_folder + survey_name + ".h5")

In [ ]:
%matplotlib widget
# for grb_name in survey:
grb_name = "SynthGRB_5"

with open(inference_folder + data_name + ".yml", "r") as f:
    d = yaml.load(f, Loader=yaml.SafeLoader)
    dets = d[grb_name]["detectors"].keys()

fig, axes = plt.subplots(2,2,sharex=True, sharey="row",figsize=set_size(width, height=width, subplots=(2,2)))
row = 0
col = 0
for k,v in survey[grb_name].grb.items():
    if k in dets:
        ax = axes[row][col]

        lightcurve = v['lightcurve']

        lightcurve.display_lightcurve(dt=.5, ax=ax,lw=1, label="Total signal")#,color='#25C68C')
        lightcurve.display_source(dt=.5,ax=ax,lw=1, label="Source")#,color="#A363DE")
        lightcurve.display_background(dt=.5,ax=ax,lw=1, label="Background")#, color="#2C342E")
        ax.set_xlim(-10, 30)
        if row != 1:
            ax.set_xlabel("")
        else:
            ax.set_xlabel(r"Time $t$ [s]")
        if col != 0:
            ax.set_ylabel("")
        else:
            ax.set_ylabel(r"Count Rate $n$")
        ax.legend()
        ax.set_title(k)

        if col < 1:
            col+=1
        else:
            row+=1
            col=0

fig.align_ylabels(axes)
fig.tight_layout()

## Corner Plot

In [ ]:
%matplotlib widget
fig, axes = plt.subplots(3,3,figsize=set_size(width, height=width, subplots=(3,3)))

vars = ["alpha", "log_ec", "log_Nrest", "gamma"]
av.plot_pair(res, var_names=vars, coords={f"{i}_dim_0": 0 for i in vars}, ax=axes)

axes[2][0].set_xlabel(r"$\alpha$")
axes[2][1].set_xlabel(r"$\log E_C$")
axes[2][2].set_xlabel(r"$\log N_\mathrm{rest}$")

axes[0][0].set_ylabel(r"$\log E_C$")
axes[1][0].set_ylabel(r"$\log N_\mathrm{rest}$")
axes[2][0].set_ylabel(r"$\gamma$")

## Trace Plot

In [ ]:
%matplotlib widget
rows = len(vars)
fig, axes = plt.subplots(
    rows, 2, figsize=set_size(width, height=width, subplots=(rows, 2))
)

av.plot_trace(res, var_names=vars, axes=axes)

fig.align_ylabels(axes)
fig.tight_layout()